In [ ]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!pip install pyspark
!wget -q http://apache.mirrors.tds.net/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
%env PYSPARK_HADOOP_VERSION=3
!pip install pyspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
!PYSPARK_RELEASE_MIRROR=http://mirror.apache-kr.org PYSPARK_HADOOP_VERSION=3 pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Colab").getOrCreate()


In [ ]:
!pip install plotly


In [ ]:
import heapq


def read_graph_from_file(filename):
    graph = {}
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            source = int(parts[0].strip())
            target = int(parts[1].strip())
            weight = int(parts[2].strip())

    return graph

def dijkstra_length_and_path(graph, source, target):
    distances = {node: float('inf') for node in graph}
    distances[source] = 0
    priority_queue = [(0, source)]

    paths = {source: []}

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        if current_node == target:
            break

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight

            if distance < distances[neighbor]:
                distances[neighbor] = distance
                heapq.heappush(priority_queue, (distance, neighbor))
                paths[neighbor] = paths[current_node] + [current_node]

    if distances[target] == float('inf'):
        return [], float('inf')

    path = paths[target] + [target]

    return path, distances[target]


In [ ]:
graph = read_graph_from_file("/content/drive/MyDrive/Colab Notebooks/DIC Assignment-3/question1.txt")
print(graph)

In [ ]:

f = open("output_1.txt", "w")
for i in range(1,len(graph)):
  path, distance = dijkstra_length_and_path(graph, 0, i)
  f.write(f"Shortest path from node 0 to node {i}: {path}\n")
  f.write(f"Total distance: {distance}\n")
f.close()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

def main(filename):
    spark = SparkSession.builder \
        .appName("PageRank Example") \
        .getOrCreate()

    
    edges = spark.read.text(filename) \
        .rdd.map(lambda row: row.value.split(': ')) \
        .map(lambda x: (int(x[0]), list(map(int, x[1].strip('[]').split(', '))))) \
        .flatMapValues(lambda x: x) \
        .map(lambda x: (x[0], x[1]))

    
    edges_df = edges.toDF(["src", "dst"])


    from graphframes import GraphFrame
    vertices = edges_df.selectExpr("src as id").distinct()
    g = GraphFrame(vertices, edges_df)

   
    results = g.pageRank(resetProbability=0.15, maxIter=10)

    
    max_pagerank = results.vertices.orderBy('pagerank', ascending=False).first()
    min_pagerank = results.vertices.orderBy('pagerank', ascending=True).first()

    print(f"Highest PageRank: Node {max_pagerank['id']} with rank {max_pagerank['pagerank']:.4f}")
    print(f"Lowest PageRank: Node {min_pagerank['id']} with rank {min_pagerank['pagerank']:.4f}")

    spark.stop()

if __name__ == "__main__":
    main('/content/drive/MyDrive/Colab Notebooks/DIC Assignment-3/question2.txt')



In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.mirrors.tds.net/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark pyngrok


In [ ]:
from google.colab import drive
drive.mount('/content/drive')